# Import Statements

In [1]:
#importing all the necessary modules
import numpy as np
import pandas as pd
import random
from pprint import pprint

# Preprocessing the Data

In [2]:
df = pd.read_csv("wine-dataset.csv") #reading the file
#renaming the files helps later during the processing stage
df = df.rename(columns={"quality": "label"})
df = df.rename(columns={"fixed acidity": "fixed_acidity"})
df = df.rename(columns={"volatile acidity": "volatile_acidity"})
df = df.rename(columns={"citric acid": "citric_acid"})
df = df.rename(columns={"residual sugar": "residual_sugar"})
df = df.rename(columns={"free sulfur dioxide": "free_sulfur_dioxide"})
df = df.rename(columns={"total sulfur dioxide": "total_sulfur_dioxide"})

df.head() #this will show the top 5 elements of the dataset

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,label
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,0
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,0
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,0
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,0
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,0


### Checking if the Data is Pure

In [3]:
def purity_check(data):
    
    return len(np.unique(data[:, -1])) == 1

### Finding the Potential Splits and Splitting the Data

In [4]:
def get_class(data):
    
    uni_class, counts_uni_class = np.unique(data[:, -1], return_counts=True) #returns the unique classes as well as their counts
    return uni_class[counts_uni_class.argmax()] #returns the class with majority count

def get_potential_splits(data):
    
    pot_spl = dict()
    rows , cols = data.shape
    for ind in range(cols - 1):          # excluding the last column as it as label
        pot_spl[ind] = np.unique(data[:, ind])
    
    return pot_spl #returns the potential splits

def split(data, col, val):
    #splits the data on the basis of particular column and value
    left_split = data[data[:, col] <= val] #data with values less 
    right_split = data[data[:, col] >  val] #data with values more
    
    return left_split, right_split

### Function for Calculating the entropy and Gini Index

In [5]:
def calc_entropy(data):
    
    classes , counts = np.unique(data[:, -1], return_counts=True)
    prob = counts / counts.sum() #calc prob for each of the unique classes
    entropy = sum(prob * -np.log2(prob)) #entropy calc formula
     
    return entropy

def gini(data):
    
    impurity=1
    _, counts = np.unique(data[:, -1], return_counts=True)
    prob = counts / counts.sum() #calc prob for each of the unique classes
    for x in prob:
        impurity -= x**2 #gini index calc formula
     
    return impurity

### Function for Calculating the Information Gain and Finding best splits possible

In [6]:
def calculate_info_gain_gini(left_split, right_split,uncertainity):
    
    total=len(left_split) + len(right_split) #total number of records
    #calc prob for the splits
    prob_left = len(left_split) / total
    prob_right = len(right_split) / total

    return uncertainity -  (prob_left * gini(left_split) + prob_right * gini(right_split))

def calculate_info_gain_entropy(left_split, right_split ,uncertainity):
    
    total=len(left_split) + len(right_split) #total number of records
    #calc prob for the splits
    prob_left = len(left_split) / total
    prob_right = len(right_split) / total

    return uncertainity -  (prob_left * calc_entropy(left_split) + prob_right * calc_entropy(right_split))

def find_best_split(data, pot_spl, method):
    
    if method==0: #if method = 0 means we want entropy
        uncertainity=calc_entropy(data)
    else:
        uncertainity=gini(data) #if method = 1 means we want gini index
        
    overall_info_gain = 0
    for col in pot_spl:
        for val in pot_spl[col]:
            left_split, right_split = split(data, col, val)
            if method==0: #entropy
                current_info_gain = calculate_info_gain_entropy(left_split, right_split, uncertainity)
            else: #gini index
                current_info_gain = calculate_info_gain_gini(left_split, right_split, uncertainity)

            if current_info_gain > overall_info_gain:
                overall_info_gain = current_info_gain
                best_col = col
                best_val = val
    
    return best_col, best_val

# Main Decision Tree Algorithm

### Representation of a node in the Decision Tree

In [7]:
node = {"question": ["yes_answer", 
                         "no_answer"]}
column_names = df.columns

### Decision Tree Algorithm without Pruning

In [1]:
def decision_tree_algorithm(df, method, count=0):
    
    data = df
    if count == 0:
        data = df.values #convert from dataframe to numpy list of lists         
    
    # base case
    if (purity_check(data)):
        return get_class(data) #return the class
  
    count += 1

    pot_spl = get_potential_splits(data)
    col, val = find_best_split(data, pot_spl,method)
    left_split, right_split = split(data, col, val)
        
        # check for empty splits
    if len(left_split) == len(right_split) == 0:
        return get_class(data)
        
        # determine the question
    attribute = column_names[col]
    ques = f'{attribute} <= {val}'
        
        # recursive calls
    success = decision_tree_algorithm(left_split,method, count)
    failure = decision_tree_algorithm(right_split,method, count)
    
    # subtree structure
    node = {ques: []}
    
    if success == failure:
        node = failure
    else:
        node[ques].append(success)
        node[ques].append(failure)
        
    return node

def classification(eg, tree):
    
    if not isinstance(tree, dict):
        return tree
    
    ques = list(tree.keys())[0]
    attribute , comparison_operator, val = ques.split()

    if eg[attribute] <= float(val):
        res = tree[ques][0]
    else:
        res = tree[ques][1]

    return classification(eg, res)

# Function for calculating the Accuracy

In [9]:
def calculate_accuracy(df, tree):

    df["predictions"] = df.apply(classification, args=(tree,), axis=1 , raw=False )
    df["comparison"] = df["predictions"] == df["label"]
    
    return df["comparison"].mean()

# K-Fold Cross Validation with and without Random Sampling

In [10]:
def KFoldCross_with_random_sampling(df,splits,method):
    accuracy_scores=[]
    fold_size=round(len(df)/splits)
    indices = df.index.tolist()
    
    for k in range(splits):
        test_indices = random.sample(population=indices, k=fold_size)
        test_data = df.loc[test_indices]
        train_data = df.drop(test_indices)
        
        tree = decision_tree_algorithm(train_data ,method)
        test_accuracy = calculate_accuracy(test_data, tree)*100
        
        accuracy_scores.append(test_accuracy)
        
    return accuracy_scores


def KFoldCross_without_random_sampling(df,splits,method):
    
    indices=df.index.tolist()
    fold_size=round(len(df)/splits)
    index=0
    accuracy_scores=[]
        
    for k in range(splits):
        test_indices=[]
        while len(test_indices)<fold_size and index < len(df):
            test_indices.append(index)
            index+=1
        
        test_data = df.loc[test_indices]
        train_data = df.drop(test_indices)
        
        tree = decision_tree_algorithm(train_data,method)
        test_accuracy = calculate_accuracy(test_data, tree)*100
        
        accuracy_scores.append(test_accuracy)
        
    return accuracy_scores

# Checking the Accuracy using K-Fold Cross Validation

In [11]:
#executing this cell may take 2-3 minutes
accuracy_without_random = KFoldCross_without_random_sampling(df,10,0)#0 means using entropy
print("Accuracy using KFold without Random Sampling: {}".format(np.mean(accuracy_without_random)))

accuracy_with_random = KFoldCross_with_random_sampling(df,10,0) #0 means using entropy 
print("Accuracy using KFold with Random Sampling: {}".format(np.mean(accuracy_with_random)))

Accuracy using KFold without Random Sampling: 74.90715958514554
Accuracy using KFold with Random Sampling: 84.06122448979592


# Improvement Strategies

## 1. Using Gini Index

### The KFoldCrossVal Accuracy using Gini Index

In [12]:
accuracy_with_random = KFoldCross_with_random_sampling(df,10,1) #1 means using gini
print("Accuracy using KFold with Random Sampling and Gini: {}".format(np.mean(accuracy_with_random)))

Accuracy using KFold with Random Sampling and Gini: 83.22448979591836


# 2. Pruning

## Pruning Algorithm

In [13]:
def pruned_decision_tree_algorithm(df, method, count=0, threshold=2, max_depth=20):
    
    data = df
    if count == 0:
        data = df.values         
    
    # base case
    if (purity_check(data) | (len(data) < threshold) | (count == max_depth)): #if data is pure, threshold is reached or max_depth reached
        return get_class(data)
  
    count += 1 #incrementing the counter
    
    #determining the left and right split using the best split possible
    pot_spl = get_potential_splits(data)
    col, val = find_best_split(data, pot_spl,method)
    left_split, right_split = split(data, col, val)
        
        # check for empty splits
    if len(left_split) == len(right_split) == 0:
        return get_class(data)
        
        # determine the question
    attribute = column_names[col]
    ques = f'{attribute} <= {val}'
        
        # recursive calls
    success = pruned_decision_tree_algorithm(left_split,method, count, threshold, max_depth)
    failure = pruned_decision_tree_algorithm(right_split,method, count, threshold, max_depth)
    
     # subtree structure
    node = {ques: []}
    
    if success == failure:
        node = failure
    else:
        node[ques].append(success)
        node[ques].append(failure)
        
    return node

def KFoldCross_with_Pruning(df,splits,method):
    accuracy_scores=[]
    fold_size=round(len(df)/splits)
    indices = df.index.tolist()
    
    for k in range(splits):
        test_indices = random.sample(population=indices, k=fold_size)
        test_data = df.loc[test_indices]
        train_data = df.drop(test_indices)
        
        tree = pruned_decision_tree_algorithm(train_data ,method,max_depth=20)
        test_accuracy = calculate_accuracy(test_data, tree)*100
        
        accuracy_scores.append(test_accuracy)
        
    return accuracy_scores

### K-Fold CrossVal Accuracy using Pruning

In [14]:
#executing this cell may take 2-3 minutes
accuracy_with_pruning = KFoldCross_with_Pruning(df,10,0)
print("Accuracy using KFold with Pruning: {}".format(np.mean(accuracy_with_pruning)))

Accuracy using KFold with Pruning: 84.22448979591836


## Representation of the decision tree after Pruning(till depth 3) using Entropy and Gini

In [15]:
#method=0 means we are constructing the decision tree using entropy
print('Decision Tree Representation using Entropy')
tree=pruned_decision_tree_algorithm(df,method=0,max_depth=3)
pprint(tree)

print('\n')

#method=1 means we are constructing the decision tree using gini
print('Decision Tree Representation using Gini')
tree=pruned_decision_tree_algorithm(df,method=1,max_depth=3)
pprint(tree)

Decision Tree Representation using Entropy
{'alcohol <= 10.6': [0.0,
                     {'alcohol <= 11.7333333333333': [0.0,
                                                      {'free_sulfur_dioxide <= 21.0': [0.0,
                                                                                       1.0]}]}]}


Decision Tree Representation using Gini
{'alcohol <= 10.8': [{'volatile_acidity <= 0.2': [{'density <= 0.99784': [0.0,
                                                                          1.0]},
                                                  0.0]},
                     {'alcohol <= 12.5': [0.0,
                                          {'free_sulfur_dioxide <= 20.0': [0.0,
                                                                           1.0]}]}]}
